# Debyer Analysis
Python scripts setup in a Jupyter Notebook to enable batch calculation and statistics using the software package Debyer. Debyer is an external software package in C/C++ and needs to be seperately installed (see link below).

Use of MDScatter for PDF calculations also warrants the citation of Debyer. 

Debyer Description: "Debyer can calculate diffraction pattern (from Debye’s scattering equation), pair distribution function and a few other statistics of an atomistic model."

Debyer Link: https://debyer.readthedocs.io/en/latest/

----------

## Custom Imports
Relative import the custom classes to support the cluster network tool.

In [ ]:
# Import and run the setup script
import sys, os
import numpy as np
from typing import TypedDict, Optional, Dict

# Ensure the project root is in sys.path to locate setup_env
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# Import the setup helper and configure the environment
from setup_env import setup_environment

# Capture the imported classes from setup_environment
calculate_number_density, DebyerBatch, prunePDB, BulkVolumeParams, BulkVolume, RadiusOfGyrationCalculator, PDBEditor, TrajectoryProcessor, PDBFileHandler, Atom, ClusterNetwork, ClusterBatchAnalyzer = setup_environment()

## Batch Calculate Total G(r)
modes: 
- RDF: Calculate the radial distribution function R(r)
- rPDF: Calcualte the 'reduced' PDF G(r), some might call this 'big' G(r)
- PDF: Calculate the PDF g(r), some might call this 'little or lil' g(r) [gee-of-arre] (except nobody actually calls it that).


In [ ]:
input_folder = 'input/directory/here'
output_folder = 'output/directory/here'
filename_prefix = 'samplename_numxx_'
mode = 'PDF'
from_value=0.5
to_value=30
step_value=0.01
ro_value=0.05   ## number density value
# cutoff_value=None
bounding_box=[50, 50, 50]   ## a, b, c box dimensions

debyer = DebyerBatch(input_folder, 
                    output_folder, 
                    filename_prefix, 
                    mode=mode, 
                    from_value=from_value, 
                    to_value=to_value, 
                    step_value=step_value, 
                    ro_value=ro_value, 
                    cutoff_value=None, 
                    bounding_box=bounding_box)

## Batch Calculate Partial G(r)

### Prune PDB Files 
Prune based on residue name to conduct a partial G(r) analysis

In [ ]:
input_folder = '/Users/keithwhite/repos/MDScatter/md_sim/md_0924/PbI2_DMSO_0.8M_cr_nb_new/splitpdb'
allowed_residues = ['DMS']

dms_dms = prunePDB(input_folder, 
                     allowed_residues=allowed_residues)

dms_dms.process_all_files()

## Run Debyer

In [ ]:
# input_folder = dms_dms.xyz_folder
input_folder = '/Users/keithwhite/repos/MDScatter/md_sim/md_0924/PbI2_DMSO_0.8M_cr_nb_new/splitpdb/converted_xyzs_pruned_DMS'
output_folder = '/Users/keithwhite/repos/MDScatter/md_sim/md_0924/PbI2_DMSO_0.8M_cr_nb_new'
filename_prefix = 'pbi2dmso_0p8m_dmsdms'
mode = 'rPDF'
from_value=0.5
to_value=30
step_value=0.01
# cutoff_value=None

bounding_box=[53.764, 53.764, 53.764]   ## a, b, c box dimensions
ro_value = calculate_number_density(atom_count=10660, box_dimensions=bounding_box)

debyer = DebyerBatch(input_folder, 
                    output_folder, 
                    filename_prefix, 
                    mode=mode, 
                    from_value=from_value, 
                    to_value=to_value, 
                    step_value=step_value, 
                    ro_value=ro_value, 
                    cutoff_value=None, 
                    bounding_box=bounding_box)

In [ ]:
debyer.run_debyer()